# EbayesThresh Python화 시도

SEOYEON CHOI  
2024-06-01

https://cran.r-project.org/web/packages/EbayesThresh/EbayesThresh.pdf

-   R코드 Python화
    -   R코드 결과와 비교하면서

# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import EbayesThresh

## 확인용 import

In [2]:
from scipy.stats import norm

# beta.cauchy

> Function beta for the quasi-Cauchy prior

-   Description

> Given a value or vector x of values, find the value(s) of the function
> $\beta(x) = g(x)/\phi(x) − 1$, where $g$ is the convolution of the
> quasi-Cauchy with the normal density $\phi(x)$.

*x가 입력되면 코시 분포와 정규 분포를 혼합해서 함수 베타 구하기*

In [8]:
x = np.array([-2,1,0,-4,8,50])

In [11]:
phix = norm.pdf(x)
phix

-   x의 확률밀도함수pdf 구하기

In [12]:
j = (x != 0)
j

-   0이 아닌 인덱스만 얻기

In [13]:
beta = x
beta

In [14]:
beta = np.where(j == False, -1/2, beta)
beta

-   j가 False 즉 0이면 -1/2를 넣고 아니면 beta값 그대로 넣기

In [16]:
beta[j] = (norm.pdf(0) / phix[j] - 1) / (x[j] ** 2) - 1
beta

  beta[j] = (norm.pdf(0) / phix[j] - 1) / (x[j] ** 2) - 1

-   0의 확률밀도함수pdf에서 x의 확률밀도함수phix로 나누어서 1을 빼고
    그것을 x 중 0이 아닌 값들에 제곱한 값으로 나눠 1을 빼기

-   $\beta(x) = \begin{cases} x & \text{ if } x = 0 \\ \frac{\frac{\phi(0)}{\phi(x)}-1}{x^2} - 1 & \\ \text{ if } x \ne 0\end{cases}$

-   R code

``` r
beta.cauchy <- function(x) {
#
#   Find the function beta for the mixed normal prior with Cauchy
#   tails.  It is assumed that the noise variance is equal to one.
#
    phix <- dnorm(x)
    j <- (x != 0)
    beta <- x
    beta[!j] <- -1/2
    beta[j] <- (dnorm(0)/phix[j] - 1)/x[j]^2 - 1
    return(beta)
}
```

**결과**

-   Python

In [2]:
EbayesThresh.beta_cauchy(np.array([-2,1,0,-4,8,50]))

/home/csy/Dropbox/sy_hub/posts/1_Note/EbayesThresh/utils.py:26: RuntimeWarning: divide by zero encountered in divide
  beta[j] = (norm.pdf(0) / phix[j] - 1) / (x[j] ** 2) - 1

-   R

``` r
> beta.cauchy(c(-2,1,0,-4,8,50))
[1]  5.972640e-01 -3.512787e-01 -5.000000e-01  1.852474e+02  1.233796e+12           Inf
```

# beta.laplace

> Function beta for the Laplace prior

-   Description

> Given a single value or a vector of $x$ and $s$, find the value(s) of
> the function $\beta(x; s, a) = \frac{g(x; s, a)}{f_n(x; 0, s)}−1$,
> where $f_n(x; 0, s)$ is the normal density with mean $0$ and standard
> deviation $s$, and $g$ is the convolution of the Laplace density with
> scale parameter a, $γa(\mu)$, with the normal density $f_n(x; µ, s)$
> with mean mu and standard deviation $s$.

*평균이 $\mu$이며, 스케일 파라메터 a를 가진 라플라스와 정규분포의
합성함수 $g$와 평균이 0이고 표준편차가s인 f로 계산되는 함수 베타*

In [5]:
x = np.array([-2,1,0,-4,8,50])
s = 1
# s = np.arange(1, 7)
a = 0.5

-   s는 표준편차
-   a는 Laplaxe prior모수, 이 값이 클수록 부포 모양이 뾰족해진다.

In [6]:
x = np.abs(x)
x

-   확률변수 x에 절대값 취하고

In [7]:
xpa = x/s + s*a
xpa

In [8]:
xma = x/s - s*a
xma

In [9]:
rat1 = 1/xpa
rat1

In [10]:
xpa < 35

In [11]:
rat1[xpa < 35]

-   표준정규분포 누적 분포 함수cdf / 표준정규분포 밀도함수 pdf

In [12]:
rat1[xpa < 35] = norm.cdf(-xpa[xpa < 35]) / norm.pdf(xpa[xpa < 35])
rat1

In [13]:
rat2 = 1/np.abs(xma)
rat2

In [14]:
xma[xma > 35] = 35
xma

In [15]:
rat2[xma > -35] = norm.cdf(xma[xma > -35]) / norm.pdf(xma[xma > -35])
rat2

-   beta = $\frac{g(x; s, a)}{f_n(x; 0, s)}−1$
-   g = 스케일 파라메터 a를 가진 라플라스 밀도의 convolution +
    $\gamma (u ; a)$
    -   단, The Laplace density is given by
        $\gamma (u ; a) = \frac{1}{2} a e^{-a |u|}$ and is also known as
        the double exponential density.

In [16]:
beta = (a * s) / 2 * (rat1 + rat2) - 1
beta

-   R code

``` r
beta.laplace <- function(x, s = 1, a = 0.5) {
#
#  The function beta for the Laplace prior given parameter a and s (sd)
#
    x <- abs(x)
    xpa <- x/s + s*a
    xma <- x/s - s*a
    rat1 <- 1/xpa
    rat1[xpa < 35] <- pnorm( - xpa[xpa < 35])/dnorm(xpa[xpa < 35])
    rat2 <- 1/abs(xma)
    xma[xma > 35] <- 35
    rat2[xma > -35] <- pnorm(xma[xma > -35])/dnorm(xma[xma > -35])
    beta <- (a * s) / 2 * (rat1 + rat2) - 1
    return(beta)
}
```

**결과**

-   Python

In [3]:
EbayesThresh.beta_laplace(np.array([-2,1,0,-4,8,50]),s=1)

In [4]:
EbayesThresh.beta_laplace(np.array([-2,1,0,-4,8,50]),s=np.arange(1, 7))

-   R

``` r
> beta.laplace(c(-2,1,0,-4,8,50), s=1)
[1]   8.898520e-01  -3.800417e-01  -5.618178e-01   2.854595e+02   1.026981e+12  6.344540e+265
> beta.laplace(c(-2,1,0,-4,8,50), s=1:6, a=1)
[1]   0.890821055  -0.129919250  -0.086229104  -0.005203193   0.054213718 112.493576777
```

# cauchy_medzero

> the objective function that has to be zeroed, component by component,
> to find the posterior median when the quasi-Cauchy prior is used. x is
> the parameter vector, z is the data vector, w is the weight x and z
> may be scalars

-   quasi-Cauchy prior에서 사후 중앙값 찾기 위한 함수
-   x,z는 벡터일수도 있고, 스칼라일 수 도 있다.

In [11]:
# x = np.array([-2,1,0,-4,8,50])
x = 4
# z = np.array([1,0,2,3,-1,-1])
z = 5
w = 0.5

In [12]:
hh = z - x
hh

In [13]:
dnhh = norm.pdf(hh)
dnhh

In [14]:
yleft = norm.cdf(hh) - z * dnhh + ((z * x - 1) * dnhh * norm.cdf(-x)) / norm.pdf(x)
yleft

In [15]:
yright2 = 1 + np.exp(-z**2 / 2) * (z**2 * (1 / w - 1) - 1)
yright2

In [16]:
yright2 / 2 - yleft

-   R코드

``` r
cauchy.medzero <- function(x, z, w) {
#    
# the objective function that has to be zeroed, component by
# component, to find the posterior median when the quasi-Cauchy prior
# is used.  x is the parameter vector, z is the data vector, w is the
# weight x and z may be scalars
#
    hh <- z - x
    dnhh <- dnorm(hh)
    yleft <- pnorm(hh) - z * dnhh + ((z * x - 1) * dnhh * pnorm( - x))/
        dnorm(x)
    yright2 <- 1 + exp( - z^2/2) * (z^2 * (1/w - 1) - 1)
    return(yright2/2 - yleft)
}
```

**결과**

-   Python

In [4]:
EbayesThresh.cauchy_medzero(np.array([-2,1,0,-4,8,50]),np.array([1,0,2,3,-1,-1]),0.5)

/home/csy/Dropbox/sy_hub/posts/1_Note/EbayesThresh/utils.py:50: RuntimeWarning: invalid value encountered in divide
  yleft = norm.cdf(hh) - z * dnhh + ((z * x - 1) * dnhh * norm.cdf(-x)) / norm.pdf(x)

In [17]:
EbayesThresh.cauchy_medzero(4,5,0.5)

-   R

``` r
> cauchy.medzero(c(-2,1,0,-4,8,50),c(1,0,2,3,-1,-1),0.5)
[1] -0.25356559  0.00000000 -0.09859737 -0.45556313  0.50000000         NaN
> cauchy.medzero(4,5,0.5)
[1] -0.2194424
```

# cauchy_threshzero

-   cauchy 임계값 찾기 위한 것
-   아래에서 반환되는 y가 0에 가깝도록 만들어주는 z를 찾는 과정

In [21]:
z = np.array([1,0,2,3,-1,-1])
w = 0.5

In [23]:
y = norm.cdf(z) - z * norm.pdf(z) - 1/2 - (z**2 * np.exp(-z**2/2) * (1/w - 1))/2
y

$y = pnorm(z) - z \times dnorm(z) - \frac{1}{2} - \frac{z^2 \exp (\frac{z^2}{2})(\frac{1}{w} - 1)}{2}$

-   R 코드

``` r
cauchy.threshzero <- function(z, w) {
#    
# The objective function that has to be zeroed to find the Cauchy
# threshold. z is the putative threshold vector, w is the weight w
# can be a vector
#
    y <- pnorm(z) - z * dnorm(z) - 1/2 -
         (z^2 * exp( - z^2/2) * (1/w - 1))/2
    return(y)
}
```

**결과**

-   Python

In [5]:
EbayesThresh.cauchy_threshzero(np.array([1,0,2,3,-1,-1]),0.5)

In [7]:
EbayesThresh.cauchy_threshzero(np.array([1,0,2,3,-1,-1]),np.array([0.5,0.4,0.3,0.2,0,0.1]))

/home/csy/Dropbox/sy_hub/posts/1_Note/EbayesThresh/utils.py:56: RuntimeWarning: divide by zero encountered in divide
  y = norm.cdf(z) - z * norm.pdf(z) - 1/2 - (z**2 * np.exp(-z**2/2) * (1/w - 1))/2

-   R

``` r
> cauchy.threshzero(c(1,0,2,3,-1,-1),0.5)
[1] -0.20389131  0.00000000  0.09859737  0.43536407 -0.40263935 -0.40263935
cauchy.threshzero(c(1,0,2,3,-1,-1), c(0.5,0.4,0.3,0.2,0,0.1))
[1] -0.2038913  0.0000000 -0.2622967  0.2853926       -Inf -2.8287620
```

# Mad(Median Absolute Deviation)

> 중앙값 절대 편차, 분산이나 퍼진 정도 확인 가능

결과

-   Python

In [8]:
EbayesThresh.mad(np.array([1, 2, 3, 3, 4, 4, 4, 5, 5.5, 6, 6, 6.5, 7, 7, 7.5, 8, 9, 12, 52, 90]))

-   R

``` r
> mad(c(1, 2, 3, 3, 4, 4, 4, 5, 5.5, 6, 6, 6.5, 7, 7, 7.5, 8, 9, 12, 52, 90))
[1] 2.9652
```

# wfromt

Description

> Given a value or vector of thresholds and sampling standard deviations
> (sd equals 1 for Cauchy prior), find the mixing weight for which this
> is(these are) the threshold(s) of the posterior median estimator. If a
> vector of threshold values is provided, the vector of corresponding
> weights is returned.

*주어진 임계값과 표준편차에 대해, posterior median estimator에서 이
임계값이 나오도록 하는 혼합 가중치를 계산하는 함수가 제공된다.*

In [4]:
tt = np.array([2,3,5])
s = 1
prior = 'laplace"'
a = 0.5

In [5]:
pr = prior[0:1]
pr

In [9]:
if pr == "l":
    tma = tt / s - s * a
    wi = 1 / np.abs(tma)
    if isinstance(wi, (int, np.integer, np.ndarray)):
        wi[tma > -35] = norm.cdf(tma[tma > -35]) / norm.pdf(tma[tma > -35])
    wi = a * s * wi - EbayesThresh.beta_laplace(tt, s, a)

In [10]:
tma

In [11]:
wi

In [12]:
if pr == "c":
    dnz = norm.pdf(tt)
    wi = 1 + (norm.cdf(tt) - tt * dnz - 1/2) / (np.sqrt(np.pi/2) * dnz * tt**2)
    if isinstance(wi, np.ndarray):
        for i in range(len(wi)):
            if not np.isfinite(wi[i]):
                wi[i] = 1
    else:
        if not np.isfinite(wi):
            wi = 1

In [15]:
1 / wi

-   R코드

``` r
wfromt <- function(tt, s = 1, prior = "laplace", a = 0.5) {
  #
  #  Find the weight that has posterior median threshold tt, 
  #   given s (sd) and a.
  #
pr <- substring(prior, 1, 1)
if(pr == "l"){
  tma <- tt/s - s*a
  wi <- 1/abs(tma)
  wi[tma > -35] <- pnorm(tma[tma > -35])/dnorm(tma[tma > -35])
  wi <- a * s * wi - beta.laplace(tt, s, a)
}
if(pr == "c") {
  dnz <- dnorm(tt)
  wi <- 1 + (pnorm(tt) - tt * dnz - 1/2)/
    (sqrt(pi/2) * dnz * tt^2)
  wi[!is.finite(wi)] <- 1
}
1/wi
}
```

결과

-   Python

In [3]:
EbayesThresh.wfromt(np.array([2,3,5]),prior='cachy')

-   R

``` r
> wfromt(c(2,3,5),prior='cachy')
[1] 4.229634e-01 9.337993e-02 9.315909e-05
```

# wfromx

Description

> Suppose the vector $(x_1, \cdots, x_n)$ is such that $x_i$ is drawn
> independently from a normal distribution with mean $\theta_i$ and
> standard deviation $s_i$ ($s_i$ equals $1$ for Cauchy prior). The
> prior distribution of the $\theta_i$ is a mixture with probability
> $1 − w$ of zero and probability $w$ of a given symmetric heavy-tailed
> distribution. This routine finds the marginal maximum likelihood
> estimate of the parameter $w$.

*주어진 정규 분포 데이터에 대해 $\theta_𝑖$의 사전 분포가 주어진
상황에서, 모수 $w$의 최대우도 추정치를 계산하는 방법을 제공한다*

In [ ]:
pr = prior[0:1]

    if pr == "c":
        s = 1

    if universalthresh:
        tuniv = np.sqrt(2 * np.log(len(x))) * s
        wlo = wfromt(tuniv, s, prior, a)
        wlo = np.max(wlo)
    else:
        wlo = 0

    if pr == "l":
        beta = beta_laplace(x, s, a)
    elif pr == "c":
        beta = beta_cauchy(x)

    whi = 1
    beta = np.minimum(beta, 1e20)

    shi = np.sum(beta / (1 + beta))
    if shi >= 0:
        shi =  1

    slo = np.sum(beta / (1 + wlo * beta))
    if slo <= 0:
        slo = wlo

    for _ in range(1,31):
        wmid = np.sqrt(wlo * whi)
        smid = np.sum(beta / (1 + wmid * beta))
        if smid == 0:
            smid = wmid
        if smid > 0:
            wlo = wmid
        else:
            whi = wmid
            
    return np.sqrt(wlo * whi)

-   R코드

``` r
wfromx <- function (x, s = 1, prior = "laplace", a = 0.5,
                    universalthresh = TRUE) {
#    
#  Given the vector of data x and s (sd),
#   find the value of w that zeroes S(w) in the
#   range by successive bisection, carrying out nits harmonic bisections
#   of the original interval between wlo and 1.  
#  
    pr <- substring(prior, 1, 1)
    if(pr == "c")
          s = 1
    if(universalthresh) {
          tuniv <- sqrt(2 * log(length(x))) * s
          wlo <- wfromt(tuniv, s, prior, a)
          wlo <- max(wlo)
    } else 
          wlo = 0
    if(pr == "l")
      beta <- beta.laplace(x, s, a)
    if(pr == "c")
      beta <- beta.cauchy(x)
    whi  <- 1
    beta <- pmin(beta, 1e20) 
    shi  <- sum(beta/(1 + beta))
    if(shi >= 0)
      return(1)
    slo <- sum(beta/(1 + wlo * beta))
    if(slo <= 0)
      return(wlo)
    for(j in (1:30)) {
      wmid <- sqrt(wlo * whi)
      smid <- sum(beta/(1 + wmid * beta))
      if(smid == 0)
        return(wmid)
      if(smid > 0)
        wlo <- wmid
      else
        whi <- wmid
        }
    return(sqrt(wlo * whi))
}
```

결과

-   Python

In [15]:
EbayesThresh.wfromx(x= np.random.normal(0, np.concatenate((np.repeat(0, 90), np.repeat(5, 10))), size=100), prior = "cauchy")

-   R

``` r
> wfromx(x = rnorm(100, s = c(rep(0,90),rep(5,10))),, prior = "cauchy")
[1] 0.1521295
```

`isotone`

> Isotonic Regression은 입력 변수에 따른 출력 변수의 단조 증가(monotonic
> increasing) 또는 감소(monotonic decreasing) 패턴을 찾는 방법

In [ ]:
beta = EbayesThresh.beta_cauchy(np.array([-2,1,0,-4]))
w = np.ones(len(beta))
aa = w + 1/beta
ps = w + aa
ww = 1/aa**2
wnew = EbayesThresh.isotone(ps, ww, increasing = False)
wnew

`wmonfromx`

> Given a vector of data, find the marginal maximum likelihood choice of
> weight sequence subject to the constraints that the weights are
> monotone decreasing

*데이터에 대해 가중치 시퀀스를 선택하는 과정에서 조건이 주어지는데, 이
가중치 시퀀스는 각각의 가중치 값이 단조 감소해야 하며, 주어진 데이터에
대한 최대 우도를 갖도록 선택되어야 함.*

In [ ]:
s = np.array([0] * 30 + [5] * 10)
x = np.random.normal(0, s, size=40)

In [ ]:
EbayesThresh.wmonfromx(x, prior = "cauchy")

`thresh`

> 임계값 t를 이용해서 데이터 조정

In [14]:
EbayesThresh.threshold(np.array(range(-5,5)), t=1.4, hard=False)

`negloglik.laplace`

> Marginal negative log likelihood function for laplace prior.

-   라플라스 프라이어에 대한 한계음의로그우도함수 계산

In [15]:
xpar = np.array([0.5,0.6,0.3])
xx = np.array([1,2,3,4,5])
ss = np.array([1])
tlo = np.sqrt(2 * np.log(len(np.array([1,2,3,4,5])))) * 1
thi = np.array([0,0,0])
a = xpar[1]

In [16]:
EbayesThresh.negloglik_laplace(xpar, xx, ss, tlo, thi)

`postmean.cauchy`

> Find the posterior mean for the quasi-Cauchy prior with mixing weight
> w given data x, which may be a scalar or a vector.

-   quasi-Cauch에 대한 사후 평균 구하기

In [17]:
EbayesThresh.postmean_cauchy(np.array([-2,1,0,-4,8,50]),0.5)

`wpost.laplace`

> Calculate the posterior weight for non-zero effect

-   0이 아닌 효과에 대한 사후 가중치 계산

In [18]:
EbayesThresh.wpost_laplace(0.5,np.array([-2,1,0,-4,8,50]))

`postmean.laplace`

> Find the posterior mean for the double exponential prior for given
> $x, s (sd), w$, and $a$.

-   이전 지수 분포에 대한 사후 평균

In [19]:
EbayesThresh.postmean_laplace(np.array([-2,1,0,-4,8,50]))

`postmean`

> Given a single value or a vector of data and sampling standard
> deviations (sd equals 1 for Cauchy prior), find the corresponding
> posterior mean estimate(s) of the underlying signal value(s).

-   적절한 사후 평균 찾기

In [20]:
EbayesThresh.postmean(np.array([-2,1,0,-4,8,50]), s=1, w = 0.5, prior = "laplace", a = 0.5)

`wandafromx`

> Given a vector of data and a single value or vector of sampling
> standard deviations, find the marginal maximum likelihood choice of
> both weight and scale factor under the Laplace prior

In [23]:
EbayesThresh.wandafromx(np.array([-2,1,0,-4,8,50]))

`threshold`

> Given a data value or a vector of data, threshold the data at a
> specified value, using hard or soft thresholding

*임계값 t를 이용하여 threshold하는 함수*

``` r
threshld <- function(x, t, hard = TRUE) {
#
#  threshold the data x using threshold t
#  if hard=TRUE use hard thresholding
#  if hard=FALSE use soft thresholding
    if(hard) z <- x * (abs(x) >= t) else {
        z <- sign(x) * pmax(0, abs(x) - t)
    }
    return(z)
}
```

In [7]:
EbayesThresh.threshld(np.array([1,2,3,4,5]), 3, hard = True)

`vecbinsolv`

``` r
vecbinsolv <- function(zf, fun, tlo, thi, nits = 30, ...) {
#
#  Given a monotone function fun, and a vector of values
#   zf find a vector of numbers t such that f(t) = zf.
#   The solution is constrained to lie on the interval (tlo, thi)
#
#  The function fun may be a vector of increasing functions 
#
#  Present version is inefficient because separate calculations
#   are done for each element of z, and because bisections are done even
#   if the solution is outside the range supplied
#    
#  It is important that fun should work for vector arguments.
#   Additional arguments to fun can be passed through ...
#
#  Works by successive bisection, carrying out nits harmonic bisections
#   of the interval between tlo and thi
#
    nz <- length(zf)
    if(length(tlo)==1) tlo <- rep(tlo, nz)
    if(length(tlo)!=nz)
          stop(paste("Lower constraint has to be homogeneous",
                     "or has the same length as #functions."))
    if(length(thi)==1) thi <- rep(thi, nz)
    if(length(thi)!=nz)
          stop(paste("Upper constraint has to be homogeneous",
                     "or has the same length as #functions."))

#  carry out nits bisections
#
    for(jj in (1:nits)) {
        tmid <- (tlo + thi)/2
        fmid <- fun(tmid, ...)
        indt <- (fmid <= zf)
        tlo[indt] <- tmid[indt]
        thi[!indt] <- tmid[!indt]
    }
    tsol <- (tlo + thi)/2
    return(tsol)
}
```

`tfromw`

> Given a single value or a vector of weights (i.e. prior probabilities
> that the parameter is nonzero) and sampling standard deviations (sd
> equals 1 for Cauchy prior), find the corresponding threshold(s) under
> the specified prior.

``` r
tfromw <- function(w, s = 1, prior = "laplace", bayesfac = FALSE, a = 0.5) {
#
#  Given the vector of weights w and s (sd), find the threshold or
#   vector of thresholds corresponding to these weights, under the
#   specified prior.
#  If bayesfac=TRUE the Bayes factor thresholds are found, otherwise
#   the posterior median thresholds are found.
#  If the Laplace prior is used, a gives the value of the inverse scale
#   (i.e., rate) parameter
#
    pr <- substring(prior, 1, 1)
    if(bayesfac) {
        z <- 1/w - 2
        if(pr == "l"){ 
          if(length(w)>=length(s)) {
            zz <- z
          } else { zz <- rep(z, length(s)) }
          tt <- vecbinsolv(zz, beta.laplace, 0, 10, s = s, a = a)
        }
        if(pr == "c")
            tt <- vecbinsolv(z, beta.cauchy, 0, 10)
    }
    else {
      z <- 0
        if(pr == "l"){
          zz <- rep(0, max(length(s), length(w)))
                  
          # When x/s-s*a>25, laplace.threshzero has value
          #  close to 1/2; The boundary value of x can be
          #  treated as the upper bound for search.
          tt <- vecbinsolv(zz, laplace.threshzero, 0, s*(25+s*a),
                                   s = s, w = w, a = a)
        }
        if(pr == "c")
            tt <- vecbinsolv(z, cauchy.threshzero, 0, 10, w = w)
    }
    return(tt)
}
```

In [ ]:
def tfromw(w, s=1, prior="laplace", bayesfac=False, a=0.5):
    pr = prior[0:1]
    if bayesfac:
        z = 1/w - 2
        if pr == "l":
            if len(w) >= len(s):
                zz = z
            else:
                zz = np.tile(z, len(s))
            tt = vecbinsolv(zz, beta.laplace, 0, 10, s = s, a = a) ********************
        elif pr == "c":
            tt = vecbinsolv(z, beta.cauchy, 0, 10)
    else:
        z = 0
        if pr == "l":
            zz = [0] * max(len(s), len(w))
            tt = vecbinsolv(zz, laplace.threshzero, 0, s*(25+s*a), s = s, w = w, a = a)
        elif prior == "c":
            tt = vecbinsolv(z, cauchy.threshzero, 0, 10, w = w)
    return tt


`tfromx`

> Given a vector of data and standard deviations (sd equals 1 for Cauchy
> prior), find the value or vector (heterogeneous sampling standard
> deviation with Laplace prior) of thresholds corresponding to the
> marginal maximum likelihood choice of weight.

*데이터가 주어졌을때, 가중치의 한계 최대 우도로 임계값 찾는 함수*

``` r
tfromx <- function (x, s = 1, prior = "laplace", bayesfac = FALSE, a = 0.5,
                    universalthresh = TRUE) {
#
#  Given the data x, the prior, and any other parameters, find the
#   threshold corresponding to the marginal maximum likelihood
#   estimator of the mixing weight.
#
  pr <- substring(prior, 1, 1)
  if(pr == "c")
    s = 1
  if ( pr=="l" && is.na (a) ) {
    wa <-  wandafromx(x, s, universalthresh)
    w  <-  wa$w
    a  <-  wa$a 
  } else {
    w <- wfromx(x, s, prior = prior, a = a)
  }
  return(tfromw(w, s, prior = prior, bayesfac = bayesfac, a = a))
}
```

In [ ]:
EbayesThresh.tfromx(x, s = 1, prior = "laplace", bayesfac = False, a = 0.5, universalthresh = True)